# ASR Assignment 2022-23

This notebook has been provided as a template to get you started on the assignment.  Feel free to use it for your development, or do your development directly in Python.

You can find a full description of the assignment [here](http://www.inf.ed.ac.uk/teaching/courses/asr/2022-23/coursework.pdf).

You are provided with two Python modules `observation_model.py` and `wer.py`.  The first was described in [Lab 3](https://github.com/ZhaoZeyu1995/asr_labs/blob/master/asr_lab3_4.ipynb).  The second can be used to compute the number of substitution, deletion and insertion errors between ASR output and a reference text.

It can be used as follows:

```python
import wer

my_refence = 'A B C'
my_output = 'A C C D'

wer.compute_alignment_errors(my_reference, my_output)
```

This produces a tuple $(s,d,i)$ giving counts of substitution,
deletion and insertion errors respectively - in this example (1, 0, 1).  The function accepts either two strings, as in the example above, or two lists.  Matching is case sensitive.

## Template code

Assuming that you have already made a function to generate an WFST, `create_wfst()` and a decoder class, `MyViterbiDecoder`, you can perform recognition on all the audio files as follows:


In [1]:
import glob
import os
import wer
import observation_model
import openfst_python as fst

# ... (add your code to create WFSTs and Viterbi Decoder)
import observation_model
import math
import openfst_python as fst
import time

class MyViterbiDecoder:
    
    NLL_ZERO = 1e10  # define a constant representing -log(0).  This is really infinite, but approximate
                     # it here with a very large number
    
    def __init__(self, f, audio_file_name, l):
        """Set up the decoder class with an audio file and WFST f
        """
        self.om = observation_model.ObservationModel()
        self.f = f
        self.l = l
        
        if audio_file_name:
            self.om.load_audio(audio_file_name)
        else:
            self.om.load_dummy_audio()
        
        self.initialise_decoding()

        
    def initialise_decoding(self):
        """set up the values for V_j(0) (as negative log-likelihoods)
        
        """
        
        self.V = []   # stores likelihood along best path reaching state j
        self.B = []   # stores identity of best previous state reaching state j
        self.W = []   # stores output labels sequence along arc reaching j - this removes need for 
                      # extra code to read the output sequence along the best path
        
        for t in range(self.om.observation_length()+1):
            self.V.append([self.NLL_ZERO]*self.f.num_states())
            self.B.append([-1]*self.f.num_states())
            self.W.append([[] for i in range(self.f.num_states())])  #  multiplying the empty list doesn't make multiple
        
        # The above code means that self.V[t][j] for t = 0, ... T gives the Viterbi cost
        # of state j, time t (in negative log-likelihood form)
        # Initialising the costs to NLL_ZERO effectively means zero probability    
        
        # give the WFST start state a probability of 1.0   (NLL = 0.0)
        self.V[0][self.f.start()] = 0.0
        
        # some WFSTs might have arcs with epsilon on the input (you might have already created 
        # examples of these in earlier labs) these correspond to non-emitting states, 
        # which means that we need to process them without stepping forward in time.  
        # Don't worry too much about this!  
        self.traverse_epsilon_arcs(0)        
        
    def traverse_epsilon_arcs(self, t):
        """Traverse arcs with <eps> on the input at time t
        
        These correspond to transitions that don't emit an observation
        
        We've implemented this function for you as it's slightly trickier than
        the normal case.  You might like to look at it to see what's going on, but
        don't worry if you can't fully follow it.
        
        """
        
        states_to_traverse = list(self.f.states()) # traverse all states
        while states_to_traverse:
            
            # Set i to the ID of the current state, the first 
            # item in the list (and remove it from the list)
            i = states_to_traverse.pop(0)   
        
            # don't bother traversing states which have zero probability
            if self.V[t][i] == self.NLL_ZERO:
                    continue
        
            for arc in self.f.arcs(i):
                
                if arc.ilabel == 0:     # if <eps> transition
                  
                    j = arc.nextstate   # ID of next state  
                
                    if self.V[t][j] > self.V[t][i] + float(arc.weight):
                        
                        # this means we've found a lower-cost path to
                        # state j at time t.  We might need to add it
                        # back to the processing queue.
                        self.V[t][j] = self.V[t][i] + float(arc.weight)
                        
                        # save backtrace information.  In the case of an epsilon transition, 
                        # we save the identity of the best state at t-1.  This means we may not
                        # be able to fully recover the best path, but to do otherwise would
                        # require a more complicated way of storing backtrace information
                        self.B[t][j] = self.B[t][i] 
                        
                        # and save the output labels encountered - this is a list, because
                        # there could be multiple output labels (in the case of <eps> arcs)
                        if arc.olabel != 0:
                            self.W[t][j] = self.W[t][i] + [arc.olabel]
                        else:
                            self.W[t][j] = self.W[t][i]
                        
                        if j not in states_to_traverse:
                            states_to_traverse.append(j)

    
    def forward_step(self, t):
        
        forward_computations = 0
          
        for i in self.f.states():
            
            if not self.V[t-1][i] == self.NLL_ZERO:   # no point in propagating states with zero probability
                
                for arc in self.f.arcs(i):
                    
                    if arc.ilabel != 0: # <eps> transitions don't emit an observation
                        forward_computations += 1
                        j = arc.nextstate
                        tp = float(arc.weight)  # transition prob
                        ep = -self.om.log_observation_probability(self.f.input_symbols().find(arc.ilabel), t)  # emission negative log prob
                        prob = tp + ep + self.V[t-1][i] # they're logs
                        if prob < self.V[t][j]:
                            self.V[t][j] = prob
                            self.B[t][j] = i
                            
                            # store the output labels encountered too
                            if arc.olabel !=0:
                                self.W[t][j] = [arc.olabel]
                            else:
                                self.W[t][j] = []
                                
        return forward_computations
                            
    
    def finalise_decoding(self):
        """ this incorporates the probability of terminating at each state
        """
        
        for state in self.f.states():
            final_weight = float(self.f.final(state))
            if self.V[-1][state] != self.NLL_ZERO:
                if final_weight == math.inf:
                    self.V[-1][state] = self.NLL_ZERO  # effectively says that we can't end in this state
                else:
                    self.V[-1][state] += final_weight
                    
        # get a list of all states where there was a path ending with non-zero probability
        finished = [x for x in self.V[-1] if x < self.NLL_ZERO]
        if not finished:  # if empty
            print("No path got to the end of the observations.")
        
        
    def decode(self):
        forward_computations = 0
        initial = time.time()
        self.initialise_decoding()
        t = 1
        while t <= self.om.observation_length():
            forward_computations += self.forward_step(t)
            self.traverse_epsilon_arcs(t)
            t += 1
        self.finalise_decoding()
        final = time.time()
        return (forward_computations, (final - initial))
    
    def backtrace(self):
        
        initial = time.time()
        best_final_state = self.V[-1].index(min(self.V[-1])) # argmin
        best_state_sequence = [best_final_state]
        best_out_sequence = []
        
        t = self.om.observation_length()   # ie T
        j = best_final_state
        
        while t >= 0:
            i = self.B[t][j]
            best_state_sequence.append(i)
            best_out_sequence = self.W[t][j] + best_out_sequence  # computer scientists might like
                                                                                # to make this more efficient!

            # continue the backtrace at state i, time t-1
            j = i  
            t-=1
            
        best_state_sequence.reverse()
        
        # convert the best output sequence from FST integer labels into strings
        
        
        
        # convert the best output sequence from FST integer labels into strings
        best_out_sequence = transduce_sequence_det(Ldet,[ self.f.output_symbols().find(label) for label in best_out_sequence])
        
        # convert the best output sequence from FST integer labels into strings

        final = time.time()
        return (best_state_sequence, best_out_sequence,(final-initial))
import openfst_python as fst

def parse_lexicon(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            lex[line[0]] = line[1:]  # first field the word, the rest is the phones
    return lex

def generate_symbol_tables(lexicon, n=3):
    '''
    Return word, phone and state symbol tables based on the supplied lexicon
        
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
        n (int): number of states for each phone HMM
        
    Returns:
        word_table (fst.SymbolTable): table of words
        phone_table (fst.SymbolTable): table of phones
        state_table (fst.SymbolTable): table of HMM phone-state IDs
    '''
    
    state_table = fst.SymbolTable()
    phone_table = fst.SymbolTable()
    word_table = fst.SymbolTable()
    
    # add empty <eps> symbol to all tables
    state_table.add_symbol('<eps>')
    phone_table.add_symbol('<eps>')
    word_table.add_symbol('<eps>')
    
    for word, phones  in lexicon.items():
        
        word_table.add_symbol(word)
        
        for p in phones: # for each phone
            
            phone_table.add_symbol(p)
            for i in range(1,n+1): # for each state 1 to n
                state_table.add_symbol('{}_{}'.format(p, i))
            
    return word_table, phone_table, state_table


# call these two functions
lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)
def generate_phone_wfst(f, start_state, phone, n):
    """
    Generate a WFST representing an n-state left-to-right phone HMM.
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        phone (str): the phone label 
        n (int): number of states of the HMM excluding start and end
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    
    for i in range(1, n+1):
    
        in_label = state_table.find('{}_{}'.format(phone, i))
        
        # self-loop back to current state
        f.add_arc(current_state, fst.Arc(in_label, 0, None, current_state))
        
        # transition to next state
        
        # we want to output the phone label on the final state
        # note: if outputting words instead this code should be modified
        if i == n:
            out_label = phone_table.find(phone)
        else:
            out_label = 0   # output empty <eps> label
            
        next_state = f.add_state()
        f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_state))    
       
        current_state = next_state
    return current_state



def create_wfst(n):
    """ Generate a WFST for any word in the lexicon, composed of n-state phone WFSTs.
        This will currently output phone labels.  
    
    Args:
        
        start_state (int): the index of the first state, assumed to exist already
        word (str): the word to generate
        n (int): states per phone HMM
        
    Returns:
        the constructed WFST
    
    """
    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    for word, phones in lex.items():
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, None, current_state))
        
        for phone in phones: 
            current_state = generate_phone_wfst(f, current_state, phone, n)
        # note: new current_state is now set to the final state of the previous phone WFST
        
        f.set_final(current_state)
        f.add_arc(current_state, fst.Arc(0, 0, None, start_state))
        f.set_input_symbols(state_table)
        f.set_output_symbols(phone_table)
    return f




    


def read_transcription(wav_file):
    """
    Get the transcription corresponding to wav_file.
    """
    
    transcription_file = os.path.splitext(wav_file)[0] + '.txt'
    
    with open(transcription_file, 'r') as f:
        transcription = f.readline().strip()
    
    return transcription


f = create_wfst(3)

print([x for x in lex.values()])

def num_states(f):
    counter = 0
    for x in f.states():
        counter += 1
    return counter

def total_num_arcs(f):
    return sum(f.num_arcs(s) for s in f.states())



    

[['ey'], ['ah', 'v'], ['p', 'eh', 'k'], ['p', 'eh', 'p', 'er', 'z'], ['p', 'iy', 't', 'er'], ['p', 'ih', 'k', 't'], ['p', 'ih', 'k', 'ah', 'l', 'd'], ['p', 'ay', 'p', 'er'], ['dh', 'iy'], ['w', 'eh', 'r', 'z']]


In [2]:
#lexicon transducer
import openfst_python as fst


lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)  # we won't use state_table in this lab

def generate_L_wfst(lex):
    """ Express the lexicon in WFST form
    
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
    
    Returns:
        the constructed lexicon WFST
    
    """
    L = fst.Fst()
    
    # create a single start state
    start_state = L.add_state()
    L.set_start(start_state)
    
    for (word, pron) in lex.items():
        
        current_state = start_state
        for (i,phone) in enumerate(pron):
            next_state = L.add_state()
            
            if i == len(pron)-1:
                # add word output symbol on the final arc
                L.add_arc(current_state, fst.Arc(phone_table.find(phone), \
                                                 word_table.find(word), None, next_state))
            else:
                L.add_arc(current_state, fst.Arc(phone_table.find(phone),0, None, next_state))
            
            current_state = next_state
                          
        L.set_final(current_state)
        
    L.set_input_symbols(phone_table)
    L.set_output_symbols(word_table)  
    start_state = L.start()
    L.arcsort()
    for state in L.states():
        if float(L.final(state)) != math.inf:
            L.add_arc(state, fst.Arc(0, 0, None, start_state))
    
    return L

L = generate_L_wfst(lex)
Ldet = fst.determinize(L)

def transduce_sequence_det(f, seq):
    """Return transduced sequence given input sequence and determinized FST
    
        Args:
            f (fst.Fst()): a determinized FST
            in_seq (list[str]): the sequence of strings to transduce
            
        Returns:
            out_seq (list[str]): the sequence of transduced symbols
            """
    
    seq_len = len(seq)
    eps = f.input_symbols().find('<eps>')
    curr_state = f.start()
    output = []
    
    for i in range(seq_len):
        
        found = False
        label = f.input_symbols().find(seq[i])
        
        while found ==False:
            for arc in f.arcs(curr_state):
                
                if arc.ilabel == label:
                    if arc.olabel != 0:

                        output_symbol = f.output_symbols().find(arc.olabel)
                        output.append(output_symbol)
                        

                    curr_state = arc.nextstate
                    found = True
                    break
                if arc.ilabel == 0:
                    curr_state = arc.nextstate
                    

    final_weight = float(f.final(curr_state))
    if final_weight != math.inf: # if this is a final state
        return output
    else:  
        print("Can't transduce the sequence with provided FST")
        
        


In [5]:
!pip install pandas
import pandas as pd
results_df=[]
import wer

print(num_states(f))
print(total_num_arcs(f))

for wav_file in glob.glob('/group/teaching/asr/labs/recordings/*.wav'):    # replace path if using your own
                                                                           # audio files
    
    decoder = MyViterbiDecoder(f, wav_file, Ldet)
    
    forward_computations, decode_speed = decoder.decode()
    print(decode_speed)
    (state_path, words, backtrace_speed) = decoder.backtrace()  # you'll need to modify the backtrace() from Lab 4
    # to return the words along the best path
    print(backtrace_speed)
    
    transcription = read_transcription(wav_file)
    
    word_count = len(transcription.split())
    



    
    error_counts = wer.compute_alignment_errors(transcription, words)
    word_count = len(transcription.split())
    worderror = sum(error_counts)/word_count
         # you'll need to accumulate these to produce an overall Word Error Rate
    filename = wav_file.split('/')[-1]
    row = {
        'Filename': filename,
        'Insertions': error_counts[2],
        'Deletions': error_counts[1],
        'Substitutions': error_counts[0],
        'WER': worderror,
        'Word Count': word_count,
        'transcription':transcription,
        'Viterbi transcription':words,
        'Decode speed': decode_speed,
        'Backtrace speed': backtrace_speed,
        'Forward computations': forward_computations
    }
    print(row)
    
    # Append results to the DataFrame
    results_df.append(row)
    
results_df = pd.DataFrame(results_df)
# Save the DataFrame to a CSV file
results_df.to_csv('resulto.csv')



116
230
7.824498653411865
0.004067420959472656
{'Filename': '0000.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 0, 'WER': 0.23529411764705882, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': ['pickled', 'the', 'peter', 'piper', 'picked', 'a', 'the', 'peck', 'of', 'pickled', 'peppers', "where's", 'the', 'peck', 'of', 'pickled', 'peppers', 'peter', 'piper', 'picked', 'the'], 'Decode speed': 7.824498653411865, 'Backtrace speed': 0.004067420959472656, 'Forward computations': 240318}
3.69754958152771
0.0018551349639892578
{'Filename': '0001.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.0, 'Word Count': 7, 'transcription': 'peter picked a peck of pickled peppers', 'Viterbi transcription': ['pickled', 'the', 'picked', 'the', 'peter', 'picked', 'the', 'of', 'peck', 'of', 'pickled', 'peppers', 'the'], 'Decode speed': 3.69754958152771, 'Backtrace spee

2.5699880123138428
0.0011479854583740234
{'Filename': '0019.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 0, 'WER': 1.6666666666666667, 'Word Count': 3, 'transcription': 'peter piper picked', 'Viterbi transcription': ['the', 'peter', 'of', 'piper', 'the', 'a', 'picked', 'the'], 'Decode speed': 2.5699880123138428, 'Backtrace speed': 0.0011479854583740234, 'Forward computations': 104028}
3.5449979305267334
0.0022683143615722656
{'Filename': '0020.wav', 'Insertions': 9, 'Deletions': 0, 'Substitutions': 2, 'WER': 1.5714285714285714, 'Word Count': 7, 'transcription': "peppers picked peter piper where's the peck", 'Viterbi transcription': ['the', 'peppers', 'picked', 'the', 'picked', 'piper', 'the', 'a', 'peck', 'the', "where's", 'the', 'picked', 'the', 'picked', 'the'], 'Decode speed': 3.5449979305267334, 'Backtrace speed': 0.0022683143615722656, 'Forward computations': 141828}
5.117073059082031
0.0030236244201660156
{'Filename': '0021.wav', 'Insertions': 8, 'Deletions': 0, 'Subst

1.9375221729278564
0.0010592937469482422
{'Filename': '0038.wav', 'Insertions': 3, 'Deletions': 0, 'Substitutions': 4, 'WER': 0.875, 'Word Count': 8, 'transcription': 'peter piper picked a peck of pickled peppers', 'Viterbi transcription': ['peck', 'the', 'peter', 'picked', 'a', 'picked', 'picked', 'picked', 'pickled', 'peppers', 'the'], 'Decode speed': 1.9375221729278564, 'Backtrace speed': 0.0010592937469482422, 'Forward computations': 79038}
1.7384936809539795
0.0009546279907226562
{'Filename': '0039.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 0, 'WER': 0.14285714285714285, 'Word Count': 7, 'transcription': 'peppers pickled peter of the piper peck', 'Viterbi transcription': ['peppers', 'pickled', 'peter', 'of', 'the', 'piper', 'peck', 'the'], 'Decode speed': 1.7384936809539795, 'Backtrace speed': 0.0009546279907226562, 'Forward computations': 70008}
2.3205060958862305
0.001203298568725586
{'Filename': '0040.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER'

1.8523743152618408
0.0007109642028808594
{'Filename': '0056.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.42857142857142855, 'Word Count': 7, 'transcription': "where's the peck of peter piper peppers", 'Viterbi transcription': ['the', "where's", 'the', 'peck', 'of', 'peter', 'piper', "where's", 'the'], 'Decode speed': 1.8523743152618408, 'Backtrace speed': 0.0007109642028808594, 'Forward computations': 62868}
2.568577527999878
0.0010058879852294922
{'Filename': '0057.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 4, 'WER': 0.75, 'Word Count': 8, 'transcription': 'peter piper picked a peck of pickled peppers', 'Viterbi transcription': ['picked', 'the', 'picked', 'piper', 'pickled', 'peck', 'the', 'pickled', 'peppers', 'the'], 'Decode speed': 2.568577527999878, 'Backtrace speed': 0.0010058879852294922, 'Forward computations': 104028}
1.8495309352874756
0.0006759166717529297
{'Filename': '0058.wav', 'Insertions': 3, 'Deletions': 0, 'Substitutions': 2, 'WER':

2.840505599975586
0.001247406005859375
{'Filename': '0075.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.5555555555555556, 'Word Count': 9, 'transcription': 'piper peck of peck pickled picked pickled of peter', 'Viterbi transcription': ['the', 'piper', 'of', 'piper', 'of', 'peck', 'pickled', 'picked', 'peck', 'pickled', 'of', 'peter', 'the'], 'Decode speed': 2.840505599975586, 'Backtrace speed': 0.001247406005859375, 'Forward computations': 115368}
3.199652910232544
0.00127410888671875
{'Filename': '0076.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 4, 'WER': 0.8888888888888888, 'Word Count': 9, 'transcription': 'peck of peter peppers pickled peppers of picked peter', 'Viterbi transcription': ['the', 'peck', 'of', 'the', 'the', 'of', 'peck', "where's", 'pickled', "where's", 'of', 'picked', 'the'], 'Decode speed': 3.199652910232544, 'Backtrace speed': 0.00127410888671875, 'Forward computations': 129018}
3.6853442192077637
0.0013599395751953125
{'Filename':

...

1.837733507156372
0.0005481243133544922
{'Filename': '0242.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.2857142857142857, 'Word Count': 7, 'transcription': "where's the peck of peppers peter picked", 'Viterbi transcription': ["where's", 'of', 'peck', 'of', 'peppers', 'peter', 'picked', 'the'], 'Decode speed': 1.837733507156372, 'Backtrace speed': 0.0005481243133544922, 'Forward computations': 77148}
1.9447107315063477
0.0007162094116210938
{'Filename': '0243.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.7142857142857143, 'Word Count': 7, 'transcription': "where's the peck of peppers piper pickled", 'Viterbi transcription': ['the', "where's", 'picked', 'of', 'peck', 'of', 'peppers', 'the', 'piper', 'pickled', 'the'], 'Decode speed': 1.9447107315063477, 'Backtrace speed': 0.0007162094116210938, 'Forward computations': 80928}
3.7195420265197754
0.0008766651153564453
{'Filename': '0244.wav', 'Insertions': 7, 'Deletions': 0, 'Substitutions': 3, '

3.23168683052063
0.0022683143615722656
{'Filename': '0259.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 4, 'WER': 0.5882352941176471, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': ['peck', 'a', 'the', 'picked', 'of', 'of', 'of', 'picked', 'peck', 'of', 'pickled', 'of', 'peppers', "where's", 'the', 'peck', 'of', 'pickled', 'peppers', 'peter', 'of', 'picked', 'the'], 'Decode speed': 3.23168683052063, 'Backtrace speed': 0.0022683143615722656, 'Forward computations': 132798}
1.7607693672180176
0.000705718994140625
{'Filename': '0260.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 2, 'WER': 1.3333333333333333, 'Word Count': 6, 'transcription': 'peck of pickled peppers peter picked', 'Viterbi transcription': ['picked', 'the', 'peck', 'of', 'picked', 'of', 'peppers', 'picked', 'of', 'a', 'peter', 'the'], 'Decode speed': 1.7607693672180176, 'Backtrace speed': 0

2.3044161796569824
0.0011456012725830078
{'Filename': '0279.wav', 'Insertions': 3, 'Deletions': 1, 'Substitutions': 1, 'WER': 1.25, 'Word Count': 4, 'transcription': 'peck the peppers peck', 'Viterbi transcription': ['the', 'picked', 'of', "where's", 'peck', 'the'], 'Decode speed': 2.3044161796569824, 'Backtrace speed': 0.0011456012725830078, 'Forward computations': 93948}
3.4834492206573486
0.0017619132995605469
{'Filename': '0280.wav', 'Insertions': 3, 'Deletions': 0, 'Substitutions': 2, 'WER': 1.0, 'Word Count': 5, 'transcription': 'piper the piper pickled peppers', 'Viterbi transcription': ['the', "where's", 'the', 'piper', 'of', 'pickled', "where's", 'the'], 'Decode speed': 3.4834492206573486, 'Backtrace speed': 0.0017619132995605469, 'Forward computations': 92058}
2.300351619720459
0.0015308856964111328
{'Filename': '0281.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 4, 'WER': 1.4285714285714286, 'Word Count': 7, 'transcription': 'peter picked piper picked the picked pe

1.2604320049285889
0.0008475780487060547
{'Filename': '0299.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.2857142857142857, 'Word Count': 7, 'transcription': 'piper picked a peck of the peppers', 'Viterbi transcription': ['piper', 'picked', 'picked', 'peck', 'of', 'the', 'peppers', 'the'], 'Decode speed': 1.2604320049285889, 'Backtrace speed': 0.0008475780487060547, 'Forward computations': 50268}
1.3476262092590332
0.0007452964782714844
{'Filename': '0300.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 0, 'WER': 0.6666666666666666, 'Word Count': 3, 'transcription': "where's peter piper", 'Viterbi transcription': ['the', "where's", 'peter', 'piper', 'the'], 'Decode speed': 1.3476262092590332, 'Backtrace speed': 0.0007452964782714844, 'Forward computations': 55728}
1.0327229499816895
0.0004394054412841797
{'Filename': '0301.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 0, 'WER': 0.5, 'Word Count': 4, 'transcription': 'pickled peppers of peter', 'Vi

0.9063205718994141
0.00041413307189941406
{'Filename': '0318.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.6666666666666666, 'Word Count': 3, 'transcription': 'pickled the pickled', 'Viterbi transcription': ['peck', "where's", 'the', 'pickled'], 'Decode speed': 0.9063205718994141, 'Backtrace speed': 0.00041413307189941406, 'Forward computations': 37458}
0.958827018737793
0.0006136894226074219
{'Filename': '0319.wav', 'Insertions': 3, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.0, 'Word Count': 4, 'transcription': "of peter where's pickled", 'Viterbi transcription': ['peck', 'of', 'peter', 'picked', "where's", 'picked', "where's"], 'Decode speed': 0.958827018737793, 'Backtrace speed': 0.0006136894226074219, 'Forward computations': 39348}
2.468130111694336
0.0015096664428710938
{'Filename': '0320.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 1, 'WER': 2.3333333333333335, 'Word Count': 3, 'transcription': "where's peter piper", 'Viterbi transcription': ['t